In [0]:
#Final model for personality bases chat bot, built over a seq2seq moodel in Pytorch
#Seq2Seq code was inspired from https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#Needed to install pytorch in google colab
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x579de000 @  0x7fb0ab68c2a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [4]:
#Needed to link with google drive, if running locally with dataset file friends-final.txt , this is not needed 
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
SOS_token = 0
EOS_token = 1


class Helper:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [7]:
import pandas as pd
#Change the path to pick us friends-final.txt file provided in google drive LINK
friends_data=pd.read_csv('/content/gdrive/My Drive/friends-final.txt',sep="\t")
x=[]
y=[]

total_count = len(friends_data.index)
print (total_count)
for index,row in friends_data.iterrows():
    if (index == 0):
        prev = row['line']
        continue
    if (index != total_count-1):
        #print (friends_data.iloc[[index],[5]])
        #x.append(friends_data.iloc[[index],[5]])
        #y.append(friends_data.iloc[[index+1],[5]])
        x.append(prev)
        y.append(row['line'])
        prev = row['line'] 

def getFriendPairs(X,Y):
    pairs = []
    for i in range(len(X)):
        pairs.append(X[i]+"::"+Y[i])
    print(pairs[:5])
    return pairs

pairs = getFriendPairs(x,y)

60849
["There's nothing to tell! He's just some guy I work with!::C'mon, you're going out with the guy! There's gotta be something wrong with him!", "C'mon, you're going out with the guy! There's gotta be something wrong with him!::Alright Joey, be nice. So does he have a hump? A hump and a hairpiece?", 'Alright Joey, be nice. So does he have a hump? A hump and a hairpiece?::Wait, does he eat chalk?', "Wait, does he eat chalk?::Just, 'cause, I don't want her to go through what I went through with Carl- oh!", "Just, 'cause, I don't want her to go through what I went through with Carl- oh!::Okay, everybody relax. This is not even a date. It's just two people going out to dinner and not having sex."]


In [0]:
# # for pair in pairs:
# #     print (pair)
    
# pairs = [print(l.split("::")) for l in pairs]
# new_pair = []
# for pair in pairs:
#     pair_split = pair.split("::")
#     new_pair.append(pair_split)
# print (new_pair)

In [0]:
def readSpeakers(speak1, speak2, pairs):
    print("Reading lines...")
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split("::")] for l in pairs]
    input_speak = Helper(speak1)
    output_speak = Helper(speak2)
    return input_speak, output_speak, pairs

In [0]:
MAX_LENGTH = 30

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        len(p[0].split(' ')) > 0 and \
        len(p[1].split(' ')) > 0 and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [10]:
def prepareData(speak1, speak2, pairs):
    input_speak, output_speak, pairs = readSpeakers(speak1, speak2, pairs)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_speak.addSentence(pair[0])
        output_speak.addSentence(pair[1])
    print("Counted words:")
    print(input_speak.name, input_speak.n_words)
    print(output_speak.name, output_speak.n_words)
    return input_speak, output_speak, pairs


input_speak, output_speak, pairs = prepareData('Speaker 1', 'Speaker 2', pairs)
print(random.choice(pairs))

Reading lines...
Read 60847 sentence pairs
Trimmed to 1805 sentence pairs
Counting words...
Counted words:
Speaker 1 1984
Speaker 2 1976
['i m with hamilton !', 'he s good with rear things bring him in too .']


In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_speak, pair[0])
    target_tensor = tensorFromSentence(output_speak, pair[1])
    return (input_tensor, target_tensor)

In [0]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    #print(input_length)
    target_length = target_tensor.size(0)
    #print(target_length)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [0]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [18]:
print (pairs[20])

['joey uh omnipotent .', 'you are ? ross i m sorry . .']


In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_speak, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_speak.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [0]:
def evaluateSent(encoder, decoder, sent):
    try :
        output_words, attentions = evaluate(encoder, decoder, sent)
        output_sentence = ' '.join(output_words)
        print(">",sent)
        print ('<', output_sentence)
    except:
        print("Sentence contains a word which is not in our vocabulary")

In [22]:
print (input_speak.n_words)
print (output_speak.n_words)

1984
1976


In [23]:
hidden_size = 512
encoder1 = EncoderRNN(input_speak.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_speak.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 20000, print_every=100)

0m 9s (- 32m 36s) (100 0%) 4.3768
0m 17s (- 28m 46s) (200 1%) 3.8898
0m 24s (- 26m 54s) (300 1%) 3.4136
0m 32s (- 26m 11s) (400 2%) 3.4603
0m 38s (- 25m 4s) (500 2%) 3.0488
0m 46s (- 24m 56s) (600 3%) 3.3255
0m 53s (- 24m 41s) (700 3%) 3.2558
1m 0s (- 24m 17s) (800 4%) 3.0696
1m 8s (- 24m 12s) (900 4%) 3.4840
1m 16s (- 24m 4s) (1000 5%) 3.3250
1m 23s (- 23m 50s) (1100 5%) 2.9206
1m 30s (- 23m 34s) (1200 6%) 2.9876
1m 37s (- 23m 22s) (1300 6%) 3.1870
1m 45s (- 23m 18s) (1400 7%) 3.3092
1m 53s (- 23m 14s) (1500 7%) 3.3011
2m 0s (- 23m 4s) (1600 8%) 3.5207
2m 8s (- 22m 58s) (1700 8%) 3.1314
2m 15s (- 22m 49s) (1800 9%) 3.1835
2m 22s (- 22m 42s) (1900 9%) 3.2997
2m 30s (- 22m 31s) (2000 10%) 3.0749
2m 37s (- 22m 22s) (2100 10%) 3.1566
2m 44s (- 22m 11s) (2200 11%) 3.2103
2m 51s (- 21m 56s) (2300 11%) 3.0570
2m 58s (- 21m 47s) (2400 12%) 2.9792
3m 5s (- 21m 41s) (2500 12%) 3.2593
3m 12s (- 21m 31s) (2600 13%) 3.0443
3m 19s (- 21m 19s) (2700 13%) 2.8801
3m 27s (- 21m 17s) (2800 14%) 3.2883
3

In [0]:
torch.save(encoder1.state_dict(), "encoder1short20000")
torch.save(attn_decoder1.state_dict(), "attn_decoder1short20000")

In [26]:
evaluateSent(encoder1,attn_decoder1, "yeah .")
evaluateSent(encoder1,attn_decoder1, "how you doing")
evaluateSent(encoder1,attn_decoder1, "where is everyone .")
evaluateSent(encoder1,attn_decoder1, "who are you ?")
evaluateSent(encoder1,attn_decoder1, "when is the next one coming .")
evaluateSent(encoder1,attn_decoder1, "do you know ?")
evaluateSent(encoder1,attn_decoder1, "nice .")
evaluateSent(encoder1,attn_decoder1, "how are you .")



> yeah .
< i m gonna use it up . <EOS>
> how you doing
< i m okay go out okay ? <EOS>
> where is everyone .
< he s in a oh ! <EOS>
> who are you ?
< i m not finished my you here ross phoebe s friend from the ! i m just gonna be just fine ! <EOS>
> when is the next one coming .
< i m gonna marry you you re the best . <EOS>
> do you know ?
< i m sorry about <EOS>
> nice .
< i m so sorry . <EOS>
> how are you .
< i m fine you re i m m m i m m with you ? <EOS>


In [0]:
# encoder = torch.load("encoder1")
# #encoder.eval()

# #decoder = TheModelClass(*args, **kwargs)
# decoder =torch.load("attn_decoder1")
# evaluateSent(encoder,decoder, "yeah .")

Sentence contains a word which is not in our vocabulary


In [25]:
evaluateRandomly(encoder1, attn_decoder1)

> maybe a hello kitty doll the ability to walk . . .
= i m gonna get back to retraining .
< i m gonna get back to retraining . <EOS>

> ok . smelly cat smell ly cat what are they feeding you ? smelly cat oh woah oh my god . i mean like who was that ?
= they re your backup singers . . . beind you .
< they re your backup singers . . . . . <EOS>

> no i don t think we ll be doing that .
= we re not gonna do that .
< we re gonna do that . <EOS>

> spoon ? so what do you think ?
= i m torn between my integrity and my desire to avoid a beating . but i must be honest your soap is abysmal .
< i m on in a <EOS>

> yeah i just ordered a beer !
= you re straight . i get it .
< you re a . . . . <EOS>

> told ya .
= they are needy they are jumpy and you can t tell what they are thinking and that scares me a little bit .
< i m going to the bathroom . <EOS>

> well you might want to tell him it sounds like his wife is gay .
= she is not . . . she s gay . oh my god . she is so gay ! i can t believe th